# GWAS Tutorial for Hail-jp hands on seminar

このノートブックは、HailのドキュメントにあるGenome-Wide Association Study(GWAS) Tutorial(https://hail.is/docs/0.2/tutorials-landing.html) をベースに、Hail-jpのハンズオンセミナーのために加筆したものです。

遺伝子データセットを操作および探索する機能に重点を置いて、Hailの機能の概要を習得できるように設計されています。 
また、GWAS解析の中で、母集団の層別化によって引き起こされる交絡を制御する手順も入っています。

****

## Hailを起動 
**モジュールhailをhlとしてロードし、イニシャライズします。**

In [ ]:
import hail as hl
hl.init()

Hailのステキなアスキーアートが表示されれば準備OKです。  

**次に、ノートブックで使用するためにいくつかの標準Pythonライブラリをインポートします。**

ひとつひとつが、このノートブックの後半で使われる機能をセットアップしています。   
少し冗長に感じるかもしれませんが、以下に解説をいれておきます。


|命令文|意味、用途|
|----|----|
|from hail.plot import show| 後半の図表作成、show(p)で使われる機能を準備しています|
|from pprint import pprint|後半の結果表示で使われます。きれいに整形された出力や表示が可能です|
|hl.plot.output_notebook()|データ可視化ライブラリbokehを利用し、notebookに図表を描画できるようにしています|

In [ ]:
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

***
## サンプルデータの準備と読み込み

### Download public 1000 Genomes data

1000人ゲノムのデータ(ジェノタイピングされたSNPが入った大きなVCFファイル)を約20MBにダウンサンプリングすることによって作成された小さなデータを使用します。 また、別のテキストファイルからのサンプルメタデータとバリアントメタデータを統合していきます。

これらのファイルはHail teamにより、Google Storageでホストされています。    
次の命令を実行することで次のファイルがローカル環境にダウンロードされます。

<img src="images/hail-sample-datas-list.png" width="300">

In [ ]:
hl.utils.get_1kg('data/')

### VCFファイルをインポートします

VCFファイルをHailネイティブなデータ形式である[MatrixTable](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable)にインポートします。  
まず冒頭のhl.import_vcf('data/1kg.vcf.bgz')でインポートし、  
つづきの.write('data/1kg.mt', overwrite=True)でHailのネイティブなファイル形式で書き込んでいます。  
これにより、以降のデータ解析がはるかに高速になります。

In [ ]:
hl.import_vcf('data/1kg.vcf.bgz').write('data/1kg.mt', overwrite=True)

**次に、今書き込んだファイルを変数mtとして読み込みます**  
任意の名前をつけられますがここでは、'm'atrix 't'ableということでmtとしてみました。

hlの続きにread_matrix_tableとあるのがMatrixTableを読み込む命令です。  
詳しいオプションはHailのドキュメント 
(https://hail.is/docs/0.2/methods/impex.html#hail.methods.read_matrix_table)  
に記載があります。  
他の機能を探したいときやオプションを知りたいときにはこのあたりを参照します。


<img src="images/RefDoc-read_matrix_table.png" width="500">

In [ ]:
mt = hl.read_matrix_table('data/1kg.mt')

### データを見てみましょう

データセットを操作し、一部を確認したり、条件に応じたデータを抽出したり、要約したりといったことは簡単に実行できます。  
この辺りの機能を以下で示していきます。

**rowフィールドを覗いてみます**

メソッド[rows](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.rows) で、MatrixTableからrowフィールドを取得します。  

つづけて[select](https://hail.is/docs/0.2/hail.Table.html#hail.Table.select) メソッドで、取り出す情報の選択をしています。  
(引数として、テーブル内のフィールド名を参照する文字列や、[Hail Expression](https://hail.is/docs/0.2/hail.expr.Expression.html?#expression)
のいずれかを指定できます。)  
(ここでは、引数を空白のままにしていますが、そうするとrowフィールドのキーである `locus`と` alleles`のみを返してきます。)

つづけて`show` を実行することでnotebookに表示ができます。  
(メソッドに引数5を指定して使うことで、バリアントの最初の5つを表示させることができます。 )


In [ ]:
mt.rows().select().show(5)

もしくは、row_keyという命令を使うとRowのkeyを表示するので同じ結果が得られます。　

In [ ]:
mt.row_key.show(5)

#### 補足 rowフィールドには何が入っているの？？
**mt.rows().describe()を実行するとフィールドの概要が表示されます**  

見てみると、rowsにはvariantに関するデータがあり、locus, alleles, rsid, qual, filters, infoなどが含まれていることがわかります。


In [ ]:
mt.rows().describe()

### MatrixTableにあるサンプルIDを見てみる
サンプルIDの初めの5個を見てみます。  
sは、mtのcolumnフィールドのkeyで、サンプルIDが格納されています。
columnフィールドにはサンプルに関する情報が格納されます。

In [ ]:
mt.s.show(5)

**最初のいくつかの遺伝子型を表示させてみます**  

遺伝子型はentriesフィールドに入っていて、  
[entries](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.entries)  
を使うことで取り出せます。  

`take`を使い表示させています。  
`take` は最初のn行を収集してきます。  

もしくは、表示形式は異なりますが先ほど使った`show`でも構いません。  

In [ ]:
mt.entry.take(5)

In [ ]:
mt.entry.show(5)

***
## MatrixTable ?? rowフィールド？？ 整理してみます
ここまでで、VCFファイルをインポートして、hailのネイティブな形式であるMatrixTableにしました。  
また、MatrixTableの中を少し見てみました。  
MatrixTableではVCFファイルにある様々なデータを、計算機で扱いやすいように整理して格納していますが、もう少し詳しくみてみます。  

## MatrixTableの概要を知る
**mt.describe()を実行すると概要をみることができます**  

Global fields, Column fields, Row fields, Entry fields, Column key, Row keyがあることがわかります。  

VCFファイルにあった情報がどのフィールドに格納されているか、確認してみましょう。  

Column fieldにはサンプルの情報が、  
Row fieldはバリアントの情報が、  
Entry fieldにはサンプルごとの情報が入っていると思います。


In [ ]:
mt.describe()

**※実行してみましょう**  
次の3つのセルの、冒頭にある'#'を削除して実行してみてください。  
それぞれのフィールドの内容が表示されればOKです。

- column field

In [ ]:
#mt.cols().show(5)

- row field

In [ ]:
#mt.rows().show(5)

- entry field

In [ ]:
#mt.entries().show(5)

##### 実行できましたか？？ 

hailではこのようにして格納されたデータを参照することができます。  
また、ひとつひとつのデータについて検索したり、条件で絞り込んだりする操作ができます。   

このチュートリアルの後半ではそうした機能を使い、解析を進めていくことをみることができます。

※参考:  
[MatrixTableのCheat sheet](https://hail.is/docs/0.2/_static/cheatsheets/hail_matrix_tables_cheat_sheet.pdf)も便利ですよ。  
また、あとで出てくる[HailのTable(以下 Table)のcheat sheet](https://hail.is/docs/0.2/_static/cheatsheets/hail_tables_cheat_sheet.pdf)もあります。

### columnフィールドへデータを追加

HailのMatrixTableでは遺伝統計研究で重要なアノテーションなど、任意の新たなデータを追加していくことができます。
Columnフィールドはサンプルの表現型、祖先、性別や共変量に関する情報を格納するのに使います。  
RowフィールドはQCまたは分析で使用するための遺伝子や機能的影響などの情報を格納するために使用できます。   

**ここでは、テキストファイルをもとにMatrixTableに注釈をつける方法を示します。**

使うテキストファイルには、サンプルID(sample ID)、populationとsuper-population、性別、および2つのシミュレートされた表現型（1つはバイナリ、1つは離散）が含まれています。

このcsvファイルはHailの[import_table](https://hail.is/docs/0.2/methods/impex.html#hail.methods.import_table)で、インポートできます。  
これにより、[Table](https://hail.is/docs/0.2/hail.Table.html#hail.Table)オブジェクトが生成されます。  

データは先の手順でDownloadしている、data/1kg_annotations.txtを使います。  
データの型は、impute=Trueを使い推定させています。  
key_by('Sample')により、後ほどMatrixTableとの結合に使うキーを指定しています。

このTableはPandasやRのdataframeのようなものですが、Sparkの恩恵を受けているのでスケーラブルなものです。

In [ ]:
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))

これで読み込めましたが、一応、元のファイルの内容を確認してみます。

In [ ]:
!head data/1kg_annotations.txt

読み込んだテーブルがどのようになったか見てみましょう。  
**table.describe()で概要を確認できます。**

In [ ]:
table.describe()

**MatrixTableのときと同じように、`show`を使うことでデータを確認できます。**

In [ ]:
table.show(width=100)

次に、このテーブルを使用してMatrixTableのcolumnフィールドにアノテーションを格納してみます。   
まず、**既存のcolumnフィールドのスキーマを確認します。**

In [ ]:
mt.col.describe()

MatrixTableの機能である[annotate_cols](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_cols) メソッドを使用して、**MatrixTableにこのtableを結合します。**  
  
ここでは、mt.sにあるsample ID(HG00096など)を使い、tableの中を検索し、ヒットしたものをcolsフィールドにphenoという構造体で格納しています。

In [ ]:
mt = mt.annotate_cols(pheno = table[mt.s])

tableにあったPurpleHairなどの情報がMatrixTableでどのようになったか、確認してみましょう。　　

In [ ]:
mt.col.describe()

In [ ]:
mt.col.show()

細かいですが、↑の表記(mt.col)は [HailのStructExpression] (https://hail.is/docs/0.2/hail.expr.StructExpression.html#structexpression) です。  
続けてdescribeのほかselectなども利用可能ですが、MatrixTableでもTableでもなかったりします。  

**その出力やオブジェクトが何かを判定するには次のように.show()などを付与せずに実行するとわかります。**

In [ ]:
mt.col

***
## データの操作

### Query functions and the Hail Expression Language

Hailにはデータセットの統計情報を計算し取得するための便利な関数がいくつかあります。  
これらの関数は引数として`Hail Expression`をとります。

まず、**先ほど読み込んだtableの統計情報を取得してみましょう。**  
[aggregate](https://hail.is/docs/0.2/hail.Table.html#hail.Table.aggregate)
メソッドを使うことで、tableの行を集計できます。

下のセルで、`counter`は、一意の各要素の出現回数をカウントする集計関数です。  
これを使用して、カウントしたいフィールドを表す`Hail Expression`(ここでは、table.SuperPopulation)を渡すことにより、人口分布を集計することができます。

1. table.aggregateで集計を指示
2. 集計内容はcounter
3. 集計対象はtable.SuperPopulation  
という流れです

*参考: table.SuperPopulationの中身* 
<img src="images/table-sp-show.png" width="300">

In [ ]:
pprint(table.aggregate(hl.agg.counter(table.SuperPopulation)))

#### その他の項目でも試行してみましょう

- pprint(table.aggregate(hl.agg.counter(table.Population)))

In [ ]:
pprint(table.aggregate(hl.agg.counter(table.Population)))

- pprint(table.aggregate(hl.agg.counter(table.isFemale)))

In [ ]:
pprint(table.aggregate(hl.agg.counter(table.isFemale)))

- pprint(table.aggregate(hl.agg.counter(table.PurpleHair)))

In [ ]:
pprint(table.aggregate(hl.agg.counter(table.PurpleHair)))

- pprint(table.aggregate(hl.agg.counter(table.CaffeineConsumption)))

In [ ]:
pprint(table.aggregate(hl.agg.counter(table.CaffeineConsumption)))

#### それぞれの統計情報を取得

`stats` を使うと統計情報を取得するように集計内容が変わります。  
たとえばCaffeineConsumptionの分布を見るには、

1. table.aggregateで集計を指示
2. 集計内容はstats
3. 集計対象はtable.CaffeineConsumption  
という流れです

In [ ]:
pprint(table.aggregate(hl.agg.stats(table.CaffeineConsumption)))

良い感じですね。  

ですが実は、これらは先のVCFファイル由来のデータセット内にあるサンプルをきちんと表せているとは限りません。  
理由は次のとおりです。

**tableに.count()を付与することで、tableに登録されているデータの件数が確認できます**

In [ ]:
table.count()

**mtに.count_cols()を付与することで、mtのcolsフィールドに登録されているサンプルのデータ件数を確認できます。**  
または、mt.cols().count()でも同じことができます。

In [ ]:
mt.count_cols()

tableに比べてmtは、そもそもダウンサンプリングしていたものなので1000人ゲノムのデータにしては少ないです。  
そのため、見るべきはMatrixTable mtに付与したアノテーションの方でした。  

この場合は  
**[aggregate_cols](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.aggregate_cols) 
を使うことで、MatrixTableにあるサンプルに対するメトリクスを取得することができます。**

1. mt.aggregate_colsで、MatrixTableのcolumnフィールドより集計することを指示
2. 集計内容はcounter
3. 集計対象はmt.pheno.SuperPopulation
という流れです

In [ ]:
mt.aggregate_cols(hl.agg.counter(mt.pheno.SuperPopulation))

**同様に、CaffeineConsumptionの統計値を取得してみます。**

*参考: mt.CaffeineConsumptionの中身*   
<img src="images/mt-cc-show.png" width="300">

In [ ]:
pprint(mt.aggregate_cols(hl.agg.stats(mt.pheno.CaffeineConsumption)))

#### **その他のツールと何がちがうの？**
これらの機能は世間一般でそれほど新しいものではありません。  
そもそもPandasやRのDataFrame、あるいはawkなどのUnixツールで実施してきたことです。  
しかしHailでは、
- このような(平易な)インターフェースや言語を用いて、
- すべてのvariantのセットなどの遥かに大きなデータを分析すること  
ができるのです。  


#### **次に、12通りの一位なSNPをそれぞれカウントしてみます。**  

それには各バリアントのalternate alleleを取得し、それぞれのRef/Altペアをカウントしていく必要があります。  
これは、Hailの`counter`機能で実現できます。

先の例と同様に、今度はaggregate_rowsを使うことでバリアントの情報が格納されているrowsフィールドの集計ができます。  

1. mt.aggregate_rowsで、MatrixTableのrowsフィールドより集計することを指示
2. 集計内容はcounter
3. 集計対象はhl.Struct(ref=mt.alleles[0], alt=mt.allales[1])です。  
こうすることで出現したRef/Altをカウントしていくことができます。  


mt.alleles[0]やmt.alleles[1]は下の画像のとおり、Reference/Alternative それぞれのアレルを取り出します。


<img src="images/ref-alt-alleles.png" width="500">

では、**snpのカウントを実行してみます。**  
結果は変数snp_countsに代入し、直後に表示させています。  

In [ ]:
snp_counts = mt.aggregate_rows(hl.agg.counter(hl.Struct(ref=mt.alleles[0], alt=mt.alleles[1])))
pprint(snp_counts)


PythonのCounterクラスを使用して、降順で一覧表示してみます。  
collectionsのCounterは、任意のリストにある各要素の出現頻度を扱うのに便利な機能のひとつです。  
most_common()で頻度が多い順にリストすることができます。  

In [ ]:
from collections import Counter
counts = Counter(snp_counts)
counts.most_common()

この小さなデータセットからでも、生物学的なものを実際に発見できてますね！  
これらの頻度はペアになっていることがわかります。 C / TとG / Aは実際には同じ突然変異であり、反対側の鎖から見ただけです。 同様に、T / AとA / Tは反対の鎖の同じ突然変異です。 C / TとA / TのSNPの頻度には30倍の違いがあります。 どうして？


同じPython、R、およびUnixツールでもこの作業を行うことができますが、データの大きさの壁にぶつかり始めています。  
最新の[gnomAD release](https://gnomad.broadinstitute.org/) は、約2億5000万のバリアントを公開しています。   
そして、それはもはや単一のコンピュータのメモリに収まりません。  

遺伝子型はどうですか？   
Hailは、データセット内のすべての遺伝子型のコレクションを検索できます。これは、小さなデータセットでも大きくなっています。 284のサンプルと10,000のバリアントにより、1,000万の固有の遺伝子型が生成されます。 gnomADデータセットには、約5兆の固有の遺伝子型があります。  
  
*Apache Spark上で動作するHailを用いることで、遺伝統計研究を1台のPCやサーバーの限界から解き放つことができます。*

#### **DPのヒストグラムをプロットしてみます**
hailにはplot関数があり、次のようにMatrixTableにあるDPフィールドを直接渡すことができます。  
引数 rangeとbinsが設定されていない場合、この関数はフィールドの最小値と最大値に基づいて範囲を計算し、binはデフォルトの50を使用します。

range外のものは、'Outliters Above'や'Outliters Below'として扱われます。

*参考: mt.DPの中身*  
<img src="images/mt-dp-show.png" width="500">

In [ ]:
p = hl.plot.histogram(mt.DP, range=(0,30), bins=30, title='DP Histogram', legend='DP')
show(p)

***
## Quality Control

QCは時間を費やす必要がある場所です。反復する必要があるプロセスですが、手順は毎度異なります。QCには「これをやればOK」というソリューションは無いとも言えます。Broadでも新しいデータを得るたびに、新しい方法を見つけています。  
オープンサイエンスのマインドにのっとりQCについて議論を深めることで、ベストプラクティスを確立していくことができます。

### Sample QC
QCの基礎はデータセットの詳細を理解することにあります。  
Hailはこれを、[sample_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc) 機能によってより簡単に実行できるようにしています。  
これは、データセットの理解に有用なメトリクスを生成し、それらをcolumn フィールドに格納する機能です。

まず、**現時点でのcolumnフィールドの詳細を確認しておきます。**  
この時点ではQCに関する項目はありません。

In [ ]:
mt.col.describe()

次に、**sample_qcを実行します。**  
これによりdp_statsなどの統計値が追加されます。

In [ ]:
mt = hl.sample_qc(mt)

確認してみましょう。
sample_qcが追加されていることがわかります。

In [ ]:
mt.col.describe()

**sample_qcで得られた値をグラフにプロットしてみましょう。**  
先ほど使ったhail (hl) のplot.histogram機能でヒストグラムを描画します。  
ここではsample.qc.call_rateを対象にし、0.88から1の範囲をプロットしています。  

In [ ]:
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(p)

**同様に、gq_stats.meanを10-70の範囲でプロットします。**

*参考: mt.GQ, mt.sample_qc.gq_stats.meanの中身*  
<img src="images/mt-gq-show.png" width="400">  <img src="images/mt-sample-qc-gq-stats-mean.png" width="320">

In [ ]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(10,70), legend='Mean Sample GQ')
show(p)
# GQ: Quality of the assigned genotype. 

多くの場合、これらのメトリックには相関があります。

**hl.plot.scatterでdp_stats.meanとcall_rateの散布図を描いてみます。**  

In [ ]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
show(p)

データセットから外れ値を削除すると、通常、関連解析の結果が向上します。 任意のカットオフを作成し、それらを使用してフィルタリングできます。

ここでは**dp_stats.meanが4以上でかつ、call_rateが0.97以上であるものにフィルターしています。**  
また、どれくらいフィルターされているかを表示しています。

In [ ]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
print('After filter, %d/284 samples remain.' % mt.count_cols())

**先ほど実行した3つのグラフを再描画してみましょう。**

In [ ]:
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(p)

In [ ]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(10,70), legend='Mean Sample GQ')
show(p)

In [ ]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
show(p)

**次はGenotypeのQCです。**  
ここからしばらくは、初学者向けではなく初中級者向けの内容なので、初学者の方は分からなくても大丈夫です。

本来あるべき場所にないところを読み取っている遺伝子型を除外することをおすすめします:    
10%を超えるalternate読み取りを伴う、REF alleleのホモ接合型,  
10%を超えるreference読み取りを伴う、ALT alleleのホモ接合型,  
または、1:1に近いref/altのバランスにない、ヘテロ接合型,  
はエラーである可能性があります。  


(統計的に、1KGのような低深度のデータセットではこのメトリックを使用して悪い遺伝子型を検出することは困難です。)


以下、メモ
```
# AD: allele depth. フィルターなし状態でのカバレッジ。REF, ALTと２つの値があり、AD[1]はALTの値。
# hl.sum(mt.AD)は、REF,ALTを足した値。
# filter_condition_abの結果は、True or NA or False
# is_hom_ref: True if the call has no alternate alleles.
# is_het: True if the call contains two different alleles.
# is_hom_var: True if the call contains identical alternate alleles.
# 
# abが小さい領域はrefであるはず。大きい領域はaltであるはず。hetはその間。それ以外がエラーと仮定している。
#
# fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
# ここではTrueだった箇所の割合を算出。0.03596...
# 
# mt = mt.filter_entries(filter_condition_ab)
# ここで先のfilterを適用。entriesをカウントすると、2,719,750 -> 97,808になっている。
```

**フィルターを実行する前に、mt.entriesを表示しておきます。**  
また、あとで比較に使いたいので  
**mt.filter_rows(mt.locus == hl.Locus.parse('1:9780836')).entries().select("GT").show()  
を実行しておきます。**

In [ ]:
mt.entries().show()

In [ ]:
mt.filter_rows(mt.locus == hl.Locus.parse('1:9780836')).entries().select("GT").show()

In [ ]:
ab = mt.AD[1] / hl.sum(mt.AD)

filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))

fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
mt = mt.filter_entries(filter_condition_ab)

**上記はちょっと複雑そうに見えますので、少し細かくみてみましょう。**

- ab = md.AD[1] / hl.sum(mt.AD) の行   

<img src="images/filter-1.png" width="600">

**mt.AD[1]は、下記のmt.AD.show()で表示されるADのalt側の数値です。**  
hl.sum(mt.AD)は、ref/altの合計値です。


In [ ]:
mt.AD.show(3)

In [ ]:
mt.AD[1].show(3)

In [ ]:
hl.sum(mt.AD).show()

- filter_condition_abについて   

<img src="images/filter-2.png" width="600">

filter_condition_abの結果は、True or NA or False で返されます。  
&は論理積(AND), | "パイプ"は論理和(OR)です。  

mt.GTに続く命令は、以下のような条件でTrue or Falseを返すものです。  
```
is_hom_ref: True if the call has no alternate alleles.
is_het: True if the call contains two different alleles.
is_hom_var: True if the call contains identical alternate alleles.
```
これに、abの値によるフィルターを組み合わせています。



**filter_condition_abの内容を見てみましょう**

In [ ]:
filter_condition_ab.show(30,n_cols=100)

*1:9780836 は True, NA, Falseが都合よく混ざっているので確認に良さそうです。*   
あとで確認に使ってみます。

- fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))  について

<img src="images/filter-3.png" width="600">

ここではFalseだった箇所の割合を算出して表示していました。  


- mt = mt.filter_entries(filter_condition_ab)  

<img src="images/filter-4.png" width="600">

ここではmtのentriesフィールドにfilter_condition_abをフィルターとして適用しています。

**どのようにフィルターされたのか、1:9780836を探して見てみましょう。**

entry フィールドに条件をつけて、1:9780836のみを表示してみます。

**染色体上の位置を定義するのは次のような操作になります。**

In [ ]:
l1 = hl.Locus.parse('1:9780836')

In [ ]:
l1

**これを使ってentryフィールドにフィルターをかけてみます。**  

(ちょっと出力が多すぎるのでselectを使って結果をGTのみに限定します。)

In [ ]:
mt.filter_rows(mt.locus == l1).entries().select("GT").show(5)

ちなみに、l1のところの表記はこのようにしてもOKです。  

In [ ]:
mt.filter_rows(mt.locus == hl.Locus.parse('1:9780836')).entries().select("GT").show()

1:9780836のフィルターは↓のとおりでした。  
上のセルの結果を見るとHG00096の次がHG00254となっており、 NAやFalseとなっていたサンプルがフィルターされていることが確認できました。  

　　　　　　<img src="images/filter-c1.png" width="1400">
<img src="images/filter-c2.png" width="1800">



### Variant QC
**[variant_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc) も同様に、様々な統計情報を計算し格納します。**  
それらもプロットしたりフィルターするのに使用できます。

In [ ]:
mt = hl.variant_qc(mt)

rowsフィールドに格納されます。**確認してみましょう。**

In [ ]:
mt.row.describe()

In [ ]:
mt.rows().variant_qc.show()

variant_qcが追加されたのが確認できたと思います。  

今回のデータセットについてはフィルタリングをする必要はありませんが、通常ほとんどのデータセットでは慎重なQCが必要です。  
[filter_rows](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.filter_rows) が役にたつでしょう。

***
## Let's do a GWAS!

**まず、次のようなバリアントに絞り込みます:**  

 - common( cutoff 1%)
 - シーケンスエラーを示唆するほどハーディーワインベルク平衡[Hardy-Weinberg equilibrium](https://en.wikipedia.org/wiki/Hardy%E2%80%93Weinberg_principle) から遠くない  


まず、AF[1] (allele frequency for each ALT allele) が0.01以上であるようにフィルターします。

In [ ]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

次に、p_value_hwe (float64, p-value from two-sided test of Hardy-Weinberg equilibrium.)が 10^-6 以上であるものにフィルターします。

In [ ]:
mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6)

**現時点でのSample数とVariant数を確認します。**

In [ ]:
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

15%カットされています(最初は10000以上ありました)。ダウンサンプルする段階ですでに通常のデータセットよりcommon variantが含まれるようにしているので、この結果は一般的ではないかもしれません。

Hailでの関連解析では、columnフィールドのサンプルの表現型や共変量を使います。  
すでに関心のある表現型をデータセットにいれているので、それを使います。  


以下では**linear_regression_rows[https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression_rows] を使い、  
CaffeineConsumptionを使ったテストを行っています。**

GT.n_alt_alleles: Alternate alleleの値。0,1,2 or NA   
結果にはp_valueなどが含まれていきます。

In [ ]:
gwas = hl.linear_regression_rows(y=mt.pheno.CaffeineConsumption, 
                                 x=mt.GT.n_alt_alleles(), 
                                 covariates=[1.0])
gwas.row.describe()


線形回帰により、beta、標準誤差(standard error )、t統計量、およびp値の新しい行フィールドが追加されていることがわかります。

Hailでは可視化を簡単に実行できます。  
**[Manhattan plot](https://en.wikipedia.org/wiki/Manhattan_plot) をつくってみましょう！**

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

ちょっとよくわからないのが出てきましたね...  

**[Q-Q (quantile-quantile) plot](https://en.wikipedia.org/wiki/Q–Q_plot) を使ってgwasが適切だったのか確認してみましょう。**

In [ ]:
p = hl.plot.qq(gwas.p_value)
show(p)

### 交絡

観測されたp値は、すぐに期待値から外れています。私たちのデータセット内のすべてのSNPは、カフェインの消費に因果関係があるか（ありそうもない）、または交絡因子があります。

実は、サンプルの祖先を使用してこの表現型をシミュレートしていました。  
これは、表現型の分布の層別化につながります。  
解決策は、回帰の共変量として祖先を含めることです。  

[linear_regression_rows]（https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression_rows） 関数は、共変量として列フィールドを取ることもできます。  
報告された祖先でサンプルにすでに注釈を付けましたが、人為的ミスがあるかもしれないのでこのラベルを懐疑的にとらえてみます。  
ゲノムにはその問題はありません！  
報告された祖先を使用する代わりに、計算された主成分をモデルに含めることにより、遺伝的祖先を導入します。  

[pca]（https://hail.is/docs/0.2/methods/stats.html#hail.methods.pca） 関数は、固有値をリストとして生成し、サンプルのPCをテーブルに出力します。  
また、オプションでバリアントの負荷値を生成することもできます。  
[hwe_normalized_pca]（https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca） 関数は、PCAにHWEで正規化された遺伝子型を使用して同じことを行います。 

**hwe_normalized_pcaを使用してサンプルのPCを取得します**

#### hwe_normalized_pca
examples: 
```
eigenvalues, scores, loadings = hl.hwe_normalized_pca(dataset.GT, k=5)
```
    

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

固有値を見てみます。

In [ ]:
pprint(eigenvalues)

主成分スコアを見てみます。

In [ ]:
pcs.show(5, width=100)

サンプルごとに主成分が得られたので、それらをプロットすることができます。   
人類の歴史は、遺伝子データセットに強い影響を及ぼします。   
たった50MBのシーケンスデータセットを使用しても、主要な人口集団を回復できます。 

**得られた値をMatrixTableへ追加します。**

In [ ]:
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

**確認してみます。**右のほうにスクロールすると、scoresがあると思います。

In [ ]:
mt.cols().show(3)

**主成分スコアをプロットしてみます。**  
ラベルにはannotationのテキストファイルにあったもの由来の、SuperPopulationを使用しています。

In [ ]:
p = hl.plot.scatter(mt.scores[0], 
                    mt.scores[1],
                    label=mt.pheno.SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

線形回帰に戻りましょう。  
**共変量に性別、主成分 PC1, PC2, PC3を使い、線形回帰をやり直します**

In [ ]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption, 
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])

**まず、Q-Q plotをつくります。**

In [ ]:
p = hl.plot.qq(gwas.p_value)
show(p)

いいですね！  
この形状は、よくコントロールされたものによると思います。  
**マンハッタンプロットを描いてみましょう。**

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

私たちはcaffeine consumption locusを見つけました！  

(原文のまま:笑)
Now simply apply **Hail's Nature paper function** to publish the result.   
Just kidding, that function won't land until Hail 1.0!

***
## Rare variant analysis

ここでは、Hailのexpression languageを使用して、行フィールドと列フィールドの任意のプロパティでグループ化およびカウントする方法を示します。   
Hailは、シーケンスカーネルアソシエーションテスト（SKAT）も実装しています。 

https://hail.is/docs/0.2/methods/genetics.html#hail.methods.skat


In [ ]:
entries = mt.entries()
# MatrixTableからentriesフィイールドをHail tableとして取り出す

results = (entries.group_by(pop = entries.pheno.SuperPopulation, chromosome = entries.locus.contig)
      .aggregate(n_het = hl.agg.count_where(entries.GT.is_het())))

# SuperPopulation, 染色体番号でgroup化し、ヘテロである数をカウント

In [ ]:
results.show(10)

```
entries = mt.entries()
```
↑では、[MatrixTable.entries](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.entries) メソッドを使用して、MatrixTableをTable(HailのTable)に変換しました（サンプル、バリアントごとに1行あります。 ）。  
Tableでは様々なフィールドを使った集計がより簡単できます。  
(これは、レアバリアント分析のよくある最初のステップです。)

MAF(minor allele frequency)ビンと髪の色でグループ化し、GQの平均値を計算したい場合は次のようにします。

In [ ]:
#entries.show()

In [ ]:
entries = entries.annotate(maf_bin = hl.if_else(entries.info.AF[0]<0.01, "< 1%", 
                             hl.if_else(entries.info.AF[0]<0.05, "1%-5%", ">5%")))

results2 = (entries.group_by(af_bin = entries.maf_bin, purple_hair = entries.pheno.PurpleHair)
      .aggregate(mean_gq = hl.agg.stats(entries.GQ).mean, 
                 mean_dp = hl.agg.stats(entries.DP).mean))

# maf_binを追加。 info.AF[0]が< 0.01なら1%, など
# maf_binごと、purple_haiirごとにgroup化し、
# gq, dqの平均値を算出

In [ ]:
results2.show()

 任意の統計値を簡単に集計できることを示しました。  
 この特定の例がぴったりはまるとは限りませんが、このようなパターンを使用することで、まれな変動の影響を検出できます。

- 遺伝子ごとの機能的制約を推定するために、機能的カテゴリー（同義、ミスセンス、または機能喪失）ごとに遺伝子ごとのヘテロ接合遺伝子型の数を数えます
- 疾患に関与する遺伝子を検出するために、症例および対照における遺伝子あたりのシングルトン機能喪失変異の数を数える

***
## Epilogue

おつかれさまでした！  
最初のtutorialであるこのGWAS tutorialが終わりました。
さらなるHailのAPIと機能の詳細については、他のチュートリアルにも有用な情報があります。 
Hail関数のドキュメントについては、[Python API](https://hail.is/docs/0.2/api.html#python-api) を確認してください。 
自分の研究にHailを使用している場合は、[Zulip chat](https://hail.zulipchat.com) や [discussion forum](https://discuss.hail.is)でご意見をお聞かせください。

参考までに、今日のワークフローを1つのセルにまとめたものを次に示します。

In [ ]:
table = hl.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')

mt = hl.read_matrix_table('data/1kg.mt')
mt = mt.annotate_cols(pheno = table[mt.s])
mt = hl.sample_qc(mt)
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
ab = mt.AD[1] / hl.sum(mt.AD)
filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))
mt = mt.filter_entries(filter_condition_ab)
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

mt = mt.annotate_cols(scores = pcs[mt.s].scores)
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption, 
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])